# A.5 Structured and Record Arrays（结构化数组和记录数组）

我们可能注意到ndarray是一个同类的（homogeneous）数据容器，意思是说每一个元素在内存中都占用相同的字节数，这个是被dtype决定的。从表面上看，这似乎不允许我们表示异构的（heterogeneous）或表格式的数据。一个结构化的数组（structured array）其实是一个ndarray，其中每一个元素都可以被看做是C语言中的一个结构体（struct，C中的一种数据结构），或者被看做是SQL表格中的一行：

In [1]:
import numpy as np

In [2]:
dtype = [('x', np.float64), ('y', np.int32)]

In [4]:
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype)
sarr

array([( 1.5       ,  6), ( 3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

有很多方法可以指定一个结构化的dtype（参加numpy的文档）。一种典型方法是用一个由tuple组成的list，tuple的格式为`(field_name, field_data_type)`。现在，这些元素可以像是字典一样被访问：

In [5]:
sarr[0]

( 1.5, 6)

In [6]:
sarr[0]['y']

6

域名（field name）存储在dtype.names中。当我们访问一个结构化数据中的域，会返回一个对应的视图（view），因此没有产生复制体：

In [7]:
sarr['x']

array([ 1.5       ,  3.14159265])

# 1 Nested dtypes and Multidimensional Fields（嵌套dtypes和多维域）

当指定一个结构化dtype的时候，可以传入一个shape（int or tuple）：

In [8]:
dtype = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype=dtype)
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

这里x field指的是一个长度为3的数组：

In [9]:
arr[0]['x']

array([0, 0, 0])

而且访问arr['x']会返回一个二维数组，而不是像前一个例子里只返回一维数组：

In [10]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

这能让我们表示更复杂，嵌套的结构。我们也能使用嵌套dtypes来制作更复杂的结构：

In [11]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]

In [12]:
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)

In [13]:
data['x']

array([( 1.,  2.), ( 3.,  4.)],
      dtype=[('a', '<f8'), ('b', '<f4')])

In [14]:
data['y']

array([5, 6], dtype=int32)

In [15]:
data['x']['a']

array([ 1.,  3.])

pandas的DataFrame并不支持这个特性，尽管这个特性很想多层级索引（hierarchical indexing）。

# 2 Why Use Structured Arrays?（为什么使用结构化数组）

与pandas的DataFrame相比，numpy的结构化数组是一个相对低级的工具。这个工具能把一块内存解释为一个有复杂嵌套列的表格型结构。因为数组中每一个元素在内存里的比特大小都是固定的，结构化数组提供了一个非常快且高效的方法来写入和读取磁盘中的数据。

另一个常用的地方是写入一些固定长度大小的文件，来用C和C++处理。